In [76]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import pickle
from keras import optimizers
import itertools

In [77]:
df=pickle.load(open("desafio_1_data.pkl", 'rb'))

In [78]:
X=df.drop(columns='points')
y=df['points']

In [79]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
X_train.shape

(89274, 48)

In [81]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [82]:
clf = LinearRegression()

In [83]:
clf.fit(X_train, Y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [84]:
y_predict_test = clf.predict(X_test)
y_predict_train=clf.predict(X_train)

In [85]:
y_predict_test.shape,Y_test.shape

((22319,), (22319,))

In [86]:
print("Train MSE =",mean_squared_error(Y_train,y_predict_train),"\n"
      "Test MSE =",mean_squared_error(Y_test,y_predict_test)
     )
print("Train Score =",clf.score(X_train,Y_train),"\n"
      "Test Score =",clf.score(X_test,Y_test)
     )

Train MSE = 3.664766369901501 
Test MSE = 3.7902097944421165
Train Score = 0.618200423449637 
Test Score = 0.6058647072622159


In [87]:
X_train.shape[1]

48

In [95]:
def build_model_2():
    model = Sequential()
    model.add(Dense(64, activation='relu',
                    input_shape=(X_Train.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [89]:
# Armar modelo
model = build_model_2()

# Entrenamos el modelo con el training set completo
# Esta vez tomamos el número de epochs con mejor MAE en validación
model.fit(X_train,Y_train,
          epochs=20, batch_size=32, verbose=1)

Epoch 1/20
2790/2790 [==============================] - 3s 1ms/step - loss: 79.2448 - mae: 4.6785
Epoch 2/20
2790/2790 [==============================] - 3s 1ms/step - loss: 14.2305 - mae: 2.9205
Epoch 3/20
2790/2790 [==============================] - 3s 1ms/step - loss: 10.7117 - mae: 2.5762
Epoch 4/20
2790/2790 [==============================] - 4s 1ms/step - loss: 9.3154 - mae: 2.4032
Epoch 5/20
2790/2790 [==============================] - 3s 1ms/step - loss: 8.4278 - mae: 2.2979
Epoch 6/20
2790/2790 [==============================] - 4s 1ms/step - loss: 7.6925 - mae: 2.2088
Epoch 7/20
2790/2790 [==============================] - 3s 1ms/step - loss: 7.3287 - mae: 2.1431
Epoch 8/20
2790/2790 [==============================] - 3s 1ms/step - loss: 6.9761 - mae: 2.0881
Epoch 9/20
2790/2790 [==============================] - 3s 1ms/step - loss: 6.6917 - mae: 2.0439
Epoch 10/20
2790/2790 [==============================] - 3s 1ms/step - loss: 6.3472 - mae: 1.9996
Epoch 11/20
2790/2790 [===

In [90]:
test_mse_score_2, test_mae_score_2=model.evaluate(X_test, Y_test)
test_mse_score_2, test_mae_score_2

698/698 [==============================] - 1s 824us/step - loss: 4.5439 - mae: 1.6161


(4.543900012969971, 1.616093397140503)

In [91]:
Y_predict=model.predict(X_test)
Y_predict

array([[92.17701 ],
       [89.296   ],
       [92.336914],
       ...,
       [87.4649  ],
       [89.04827 ],
       [91.346176]], dtype=float32)

In [92]:
# Imprimimos el MAE obtenido esta vez
test_mse_score, test_mae_score = model.evaluate(X_test, Y_test)
test_mse_score,test_mae_score

698/698 [==============================] - 1s 944us/step - loss: 4.5439 - mae: 1.6161


(4.543900012969971, 1.616093397140503)

In [93]:
def build_model(input_shape=(28,28), layers=[64,64], optimizer='rmsprop'):
    # Instanciamos la clase del modelo secuencial
    model = Sequential()

    for l in layers[:-1]:
        model.add(Dense(units=l, activation='relu'))

    model.add(Dense(1))
    # Compilamos el modelo con el optimizador seleccionado
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    # Retornamos el modelo compilado
    return model

In [94]:
model = build_model(layers=[32, 32])

In [39]:
model.summary()

ValueError: ignored

In [24]:
# Definimos la "grilla" de parámetros que vamos a explorar
layers = [[512, 10],
          [64, 32,10]
         ]
optimizers = [optimizers.Adam(), optimizers.SGD(momentum=0.9, nesterov=True)]
combinaciones = list(itertools.product(layers, optimizers))
combinaciones

[([512, 10],
  <tensorflow.python.keras.optimizer_v2.adam.Adam at 0x7f4dd1edb160>),
 ([512, 10],
  <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD at 0x7f4dd1edb470>),
 ([64, 32, 10],
  <tensorflow.python.keras.optimizer_v2.adam.Adam at 0x7f4dd1edb160>),
 ([64, 32, 10],
  <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD at 0x7f4dd1edb470>)]

In [25]:
# Definamos algunas variables
n_splits = 3
batch_size = 256
epochs = 20
verbose = 1